### Este programa en su primera parte genera un dataset para entrenamiento de redes neuronales para predicción de posición de mecanismos de 4 barras. Utiliza libreria math para el cálculo de funciones trigonométricas la cual es funcional en micropython.
### el data set es utilizado para entrenar una red neuronal previamente creada la cual se entrena en entorno python y se generan los archivos de pesos y sesgos obtenidos de la red así como las entradas normalizadas.
### finalmente se carga una red neuronal funcional en micropython que ejecuta la función de predicción a partir de los pesos del entrenamiento y se ejecuta un ejemplo de predicción


In [3]:
import math

# Se crea una clase para generar números aleatorios que permitan generar el
# conjunto de entradas de longitudes y ángulo. Evita usar la librería random
class FloatLCG:
    def __init__(self, seed=12345):
        self.state = seed
        self.a = 1103515245
        self.c = 12345
        self.m = 2**31  # 2147483648

    def random(self):
        # Genera un número flotante entre 0 y 1
        self.state = (self.a * self.state + self.c) % self.m
        return self.state / self.m

    def uniform(self, low, high):
        # Genera un número flotante entre `low` y `high`
        return low + (high - low) * self.random()


##Programa micropython para calcular la posición de mecanismos de cuatro barras
# usando el método de Newton- Raphson

#cálculo de condición de Grashoff y tipo de mecanismo de acuerdo con clasificación de Barker
def id_grashof(L1, L2, L3, L4):

    longitudes = sorted([L1, L2, L3, L4])#organizar longitudes de menor a mayor
    s,p,q,l = longitudes #s=eslabón más corto,l=eslabón más largo, p y q longitudes intermedias
    suma_extremos = s + l
    suma_intermedias = p + q
    gr=True

    if suma_extremos < suma_intermedias:
        gr=True
        # Determinar qué eslabón es el más corto
        eslabones = [(L1, "fijo"), (L2, "entrada"),
                        (L3, "acoplador"), (L4, "salida")]
        min_eslabon = min(eslabones, key=lambda x: x[0])

        if min_eslabon[1] == "entrada":
                return "Grashof, manivela-balancín"
        elif min_eslabon[1] == "fijo":
                return "Grashof, doble manivela"
        elif min_eslabon[1] == "acoplador":
                return "Grashof, doble balancín"
        else:
                return "manivela-balancín"

    elif suma_extremos == suma_intermedias:
          gr=False
          return "NO Grashof, mecanismo plegable"
    else:
          return "NO Grashof, triple balancín"


#Función para calcular posición mediante Newton Raphson

def four_bar_analysis(L1, L2, L3, L4, theta2, max_iter=100, tol=1e-6):
    """
    Calcula los ángulos de los eslabones acoplador y de salida.
    Retorna: (theta3, theta4, near_singular, grashof)
    """
    # Verificar condición de Grashof
    grashof = id_grashof(L1, L2, L3, L4)

    # Estimación inicial para theta4 usando una ecuación de distancia entre
    #el apoyo fijo izquierdo y el punto de unión acoplador-salida

    theta4 = theta2  # Valor inicial
    for _ in range(50):
        dx = L1 - L4*math.cos(theta4) - L2*math.cos(theta2)
        dy = L4*math.sin(theta4) - L2*math.sin(theta2)
        error = dx**2 + dy**2 - L3**2
        if abs(error) < 1e-3:
            break
        df = 2*dx*(-L4*math.sin(theta4)) + 2*dy*(L4*math.cos(theta4))
        if df == 0:
            break
        theta4 -= error/df

    # Estimación inicial para theta3
    sin_theta3 = (L4*math.sin(theta4) - L2*math.sin(theta2))/L3
    cos_theta3 = (L1 - L4*math.cos(theta4) - L2*math.cos(theta2))/L3
    theta3 = math.atan2(sin_theta3, cos_theta3)

    # Método de Newton-Raphson para solución del sistema de ecuaciones de posición con detección de singularidades
    near_singular = False
    for _ in range(max_iter):
        # # Ecuación de lazo: r2 + r3 = r1 + r4 se asume theta1=0
        # Componente X: L2*cos(θ2) + L3*cos(θ3) = L1*cos(0) + L4*cos(θ4)
        # Componente Y: L2*sin(θ2) + L3*sin(θ3) = L1*sin(0) + L4*sin(θ4)
        lx = L2*math.cos(theta2) + L3*math.cos(theta3) - L4*math.cos(theta4) - L1
        ly = L2*math.sin(theta2) + L3*math.sin(theta3) - L4*math.sin(theta4)

        if abs(lx) < tol and abs(ly) < tol:
            break

        # Calcular Jacobiano del sistema de ecuaciones
        # Derivadas parciales de f respecto a theta3 y theta4
        J11 = -L3*math.sin(theta3)
        J12 = L4*math.sin(theta4)
        # Derivadas parciales de g respecto a theta3 y theta4
        J21 = L3*math.cos(theta3)
        J22 = -L4*math.cos(theta4)
        #determinante del Jacobiano
        det = J11*J22 - J12*J21

        # Verificar singularidad
        if abs(det) < 1e-3:
            near_singular = True

        if det == 0:
            return None, None, True, grashof # Matriz singular

        # Calcular incrementos
        delta3 = (-lx*J22 + ly*J12)/det
        delta4 = (lx*J21 - ly*J11)/det

        theta3 += delta3
        theta4 += delta4

        conf=0
        if theta3 is not None and theta3 < 0:
          configuracion='Cruzada'
          conf=1
        else:
          configuracion='Abierta'
        conf=0
    else:
        return None, None, near_singular, grashof

    # Verificación final de singularidad
    J11 = -L3*math.sin(theta3)
    J12 = L4*math.sin(theta4)
    J22 = -L4*math.cos(theta4)
    det = J11*J22 - (L4*math.sin(theta4))*(L3*math.cos(theta3))
    if abs(det) < 1e-3:
        near_singular = True

    return theta3, theta4, near_singular, grashof

# Generar dataset compatible en micropython

def generate_micro_dataset(num_samples, filename):

    # Crear archivo CSV
    with open(filename, 'w') as f:
        # Escribir encabezados
        f.write('L1,L2,L3,L4,theta2,theta3,theta4,grashof,near_singular,configuracion\n')


        samples_generated = 0
        rng = FloatLCG(seed=2025)
        while samples_generated < num_samples:
            # Generar longitudes aleatorias (0.5-10.0 unidades)

            L1 = rng.uniform(0.5, 10.0)
            L2 = rng.uniform(0.5, 10.0)
            L3 = rng.uniform(0.5, 10.0)
            L4 = rng.uniform(0.5, 10.0)

            # Verificar si se puede formar un mecanismo (desigualdad del triángulo)
            if (L1 + L2 + L3 < L4) or (L1 + L2 + L4 < L3) or \
               (L1 + L3 + L4 < L2) or (L2 + L3 + L4 < L1):
                continue

            # Generar ángulo de entrada aleatorio
            theta2 = rng.uniform(0, 2*math.pi)

            # Calcular posición del mecanismo
            theta3, theta4, near_singular,conf = four_bar_analysis(L1, L2, L3, L4, theta2)

            # Si no hay solución válida, saltar muestra
            if theta3 is None:
                continue

            # Determinar condición de Grashof
            gr = id_grashof(L1, L2, L3, L4)
            grashof = 1 if gr == True else 0

            #determinar configuración abierta o cerrada
            if theta3 is not None and theta3 < 0:
                configuracion = 1
            else:
                configuracion = 0

            # Escribir fila en CSV
            f.write(f'{L1:.4f},{L2:.4f},{L3:.4f},{L4:.4f},')
            f.write(f'{theta2:.4f},{theta3:.4f},{theta4:.4f},')
            f.write(f'{grashof},{1 if near_singular else 0},{configuracion}\n')


            samples_generated += 1
            if samples_generated % 10 == 0:
                print(f'Muestras: {samples_generated}/{num_samples}')

    return samples_generated

# Configuración de dataset para microcontrolador con MicroPython
NUM_SAMPLES = 200  # Tamaño manejable para microcontroladores
FILE_NAME = 'fourbar_data.csv'

# Generar dataset
print("\nGenerando dataset para mecanismo de 4 barras...")
samples_created = generate_micro_dataset(NUM_SAMPLES, FILE_NAME)
print(f"\nDataset creado: {FILE_NAME}")
print(f"Muestras generadas: {samples_created}")

# Verificar tamaño del archivo
try:
    with open(FILE_NAME, 'r') as f:
        content = f.read()
        file_size = len(content)
        print(f"Tamaño del archivo: {file_size} bytes")
        print(f"Tamaño promedio por muestra: {file_size/samples_created:.1f} bytes")
except Exception as e:
    print(f"Error al verificar tamaño: {e}")



Generando dataset para mecanismo de 4 barras...
Muestras: 10/200
Muestras: 20/200
Muestras: 30/200
Muestras: 40/200
Muestras: 50/200
Muestras: 60/200
Muestras: 70/200
Muestras: 80/200
Muestras: 90/200
Muestras: 100/200
Muestras: 110/200
Muestras: 120/200
Muestras: 130/200
Muestras: 140/200
Muestras: 150/200
Muestras: 160/200
Muestras: 170/200
Muestras: 180/200
Muestras: 190/200
Muestras: 200/200

Dataset creado: fourbar_data.csv
Muestras generadas: 200
Tamaño del archivo: 11456 bytes
Tamaño promedio por muestra: 57.3 bytes


#Importar y ajustar red neuronal del archivo de calculo de posición por ML
esta parte se ejecuta en python

In [4]:
#se importa el dataset generado

import pandas as pd
import numpy as np

# Cargar dataset
df = pd.read_csv('fourbar_data.csv')

# Seleccionar columnas relevantes
X = df[['L1', 'L2', 'L3', 'L4', 'theta2']].values.astype(np.float32)
y = df[['theta3', 'theta4', 'configuracion']].values.astype(np.float32)


normalización de datos

In [5]:
from sklearn.preprocessing import StandardScaler

scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = y.copy()
y_scaled[:, :2] = scaler_y.fit_transform(y[:, :2])  # Solo ángulos

# Guarda los parámetros de escalado para usarlos después en micropython (microcontrolador)
np.savetxt('X_mean.txt', scaler_X.mean_)
np.savetxt('X_scale.txt', scaler_X.scale_)
np.savetxt('y_mean.txt', scaler_y.mean_)
np.savetxt('y_scale.txt', scaler_y.scale_)


Red neuronal ajustada al dataset

In [6]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

class FourBarNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(5, 64)
        self.hidden2 = nn.Linear(64, 64)
        self.output_angles = nn.Linear(64, 2)
        self.output_config = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.hidden1(x))
        x = torch.relu(self.hidden2(x))
        angles = self.output_angles(x)
        config = torch.sigmoid(self.output_config(x))
        return torch.cat([angles, config], dim=1)

def combined_loss(pred, target):
    mse = nn.MSELoss()
    bce = nn.BCELoss()
    # Ángulos
    loss_angles = mse(pred[:, :2], target[:, :2])
    # Configuración (última columna)
    loss_config = bce(pred[:, 2], target[:, 2])
    return loss_angles + loss_config

# Preparar tensores
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y_scaled, dtype=torch.float32)

# Crear DataLoader
dataset = TensorDataset(X_tensor, y_tensor)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Instanciar modelo
model = FourBarNN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Entrenamiento
num_epochs = 150
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for xb, yb in loader:
        optimizer.zero_grad()
        pred = model(xb)
        loss = combined_loss(pred, yb)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item() * len(xb)
    if (epoch+1) % 20 == 0 or epoch == 0:
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss/len(dataset):.5f}")


Epoch 1/150, Loss: 1.69915
Epoch 20/150, Loss: 1.43788
Epoch 40/150, Loss: 1.24298
Epoch 60/150, Loss: 1.04676
Epoch 80/150, Loss: 0.90899
Epoch 100/150, Loss: 0.80281
Epoch 120/150, Loss: 0.73764
Epoch 140/150, Loss: 0.66413


Exportar pesos del entrenamiento a un archivo .txt para ser leido en micropython

In [7]:
def save_txt(matrix, filename):
    np.savetxt(filename, matrix, fmt='%.8f')

# Exportar capas ocultas y de salida
save_txt(model.hidden1.weight.detach().numpy(), 'w1.txt')
save_txt(model.hidden1.bias.detach().numpy(), 'b1.txt')
save_txt(model.hidden2.weight.detach().numpy(), 'w2.txt')
save_txt(model.hidden2.bias.detach().numpy(), 'b2.txt')
save_txt(model.output_angles.weight.detach().numpy(), 'w_angles.txt')
save_txt(model.output_angles.bias.detach().numpy(), 'b_angles.txt')
save_txt(model.output_config.weight.detach().numpy(), 'w_config.txt')
save_txt(model.output_config.bias.detach().numpy(), 'b_config.txt')

#desnormalización de datos
# Suponiendo pred = modelo(X_nuevo) (torch o numpy array)
theta3_pred = pred[0] * scaler_y.scale_[0] + scaler_y.mean_[0]
theta4_pred = pred[1] * scaler_y.scale_[1] + scaler_y.mean_[1]


##Uso de la red entrenada en micropython
### se crea una clase para poder ejecutar las funciones con más facilidad

In [18]:
import math

class FourBarNNMicro:
    def __init__(self, param_path=""):
        #se cargan los escaladores y los pesos obtenidos del entrenamiento de NN
        self.w1 = self.load_txt_matrix(param_path + 'w1.txt', as_matrix=True)
        self.b1 = self.load_txt_matrix(param_path + 'b1.txt')
        self.w2 = self.load_txt_matrix(param_path + 'w2.txt', as_matrix=True)
        self.b2 = self.load_txt_matrix(param_path + 'b2.txt')
        self.w_angles = self.load_txt_matrix(param_path + 'w_angles.txt', as_matrix=True)
        self.b_angles = self.load_txt_matrix(param_path + 'b_angles.txt')
        self.w_config = self.load_txt_matrix(param_path + 'w_config.txt', as_matrix=True)
        self.b_config = self.load_txt_matrix(param_path + 'b_config.txt')
        self.X_mean = self.load_txt_matrix(param_path + 'X_mean.txt')
        self.X_scale = self.load_txt_matrix(param_path + 'X_scale.txt')
        self.y_mean = self.load_txt_matrix(param_path + 'y_mean.txt')
        self.y_scale = self.load_txt_matrix(param_path + 'y_scale.txt')

    def load_txt_matrix(self, filename, as_matrix=False):
        mat = []
        with open(filename) as f:
              for line in f:
                  nums = [float(x) for x in line.strip().split()]
                  if as_matrix:
                      mat.append(nums)
                  else:
                      mat += nums
        return mat if not as_matrix else mat

    def relu(self, vec):
        return [max(0, v) for v in vec]

    def sigmoid(self, x):
        # Puede ser vector o escalar
        if isinstance(x, list):
            return [1 / (1 + math.exp(-v)) for v in x]
        return 1 / (1 + math.exp(-x))


    def matmul(self, W, x):
    # Si W es una lista de floats se convierte en una lista de listas
        if isinstance(W[0], float):
            W = [W]
        return [sum(wij * xi for wij, xi in zip(wrow, x)) for wrow in W]


    def add_bias(self, vec, b):
    # Si b es float, se convierte en lista de un elemento
        if isinstance(b, float):
            b = [b]
        return [v + bi for v, bi in zip(vec, b)]

## FUNCIÓN DE INFERENCIA(PREDICCIÓN)
    def infer(self, x_in):
        # Normalizar entrada
        x_norm = [(xi - m) / s for xi, m, s in zip(x_in, self.X_mean, self.X_scale)]
        # Primera capa
        x1 = self.relu(self.add_bias(self.matmul(self.w1, x_norm), self.b1))
        # Segunda capa
        x2 = self.relu(self.add_bias(self.matmul(self.w2, x1), self.b2))
        # Salidas ángulos (2 neuronas)
        angles_norm = self.add_bias(self.matmul(self.w_angles, x2), self.b_angles)
        # Salida configuración (1 neurona)
        config_raw = self.add_bias(self.matmul(self.w_config, x2), self.b_config)[0]
        config_prob = self.sigmoid(config_raw)

        print("angles_norm:", angles_norm)
        print("len(angles_norm):", len(angles_norm))

        # Desnormalizar ángulos
        theta3 = angles_norm[0] * self.y_scale[0] + self.y_mean[0]
        theta4 = angles_norm[1] * self.y_scale[1] + self.y_mean[1]
        return theta3, theta4, config_prob


## Ejecutar predicción, se usa un ejemplo con entradas y salidas conocidas

In [19]:
# Se ejecuta la clase
nn = FourBarNNMicro()

# Entrada de prueba: [L1, L2, L3, L4, theta2]
entrada = [5.0, 6.2, 4.1, 7.7, 1.15]

theta3_pred, theta4_pred, config_prob = nn.infer(entrada)

print("Ángulo acoplador (theta3):", theta3_pred)
print("Ángulo de salida (theta4):", theta4_pred)
print("Probabilidad configuración cruzada:", config_prob)


angles_norm: [0.27010867873008804, -0.22024972395927417]
len(angles_norm): 2
Ángulo acoplador (theta3): 10.967701073841548
Ángulo de salida (theta4): -147.8620428406172
Probabilidad configuración cruzada: 0.6069235351977081
